In [1]:
s = set()

In [2]:
s

set()

In [3]:
d = {1:"eins",2:"zwei"}

In [4]:
d

{1: 'eins', 2: 'zwei'}

In [5]:
l = [1,2,3,4,5]

In [6]:
for x in l:
    print(x)

1
2
3
4
5


In [7]:
print(x)

5


In [8]:
def square(x):
    return x**2

In [9]:
square(3)

9

In [10]:
lambda_square = lambda x: x**2

In [11]:
lambda_square(12)

144

In [12]:
list(map(lambda x: x**2, l))

[1, 4, 9, 16, 25]

In [13]:
class Person:
    def __init__(self, name):
        self.name = name
        
    def get_name(self):
        return self.name

In [14]:
hou = Person("Hou")

In [15]:
hou.name

'Hou'

In [16]:
import math
import sys
import random

class airport():
    """Implements an airport with airport code, city, country, x, y, and connections"""

    def __init__(self, code):
        """Creates airport object."""
        self.code = code
        self.conn = set([])

    def set(self, city, country, x, y):
        self.city = city
        self.country = country
        self.x = x
        self.y = y

    def fly2(self, a):
        self.conn.add(a)

    def str(self):
        return "%s, %s, %s" % (self.code, self.city, self.country)

class airports():
    """Implements a set of airports with their connections"""

    def __init__(self):
        self.d = {}  # a dictionary with code as key and airport as value
        self.city = {}  # for each city a set of its airports
        self.country = {}  # for each country a set of its airports
        self.goodCodes = self.codesWithRoutes(self.codesWithAirportInfo())
        self.readRoutes()
        self.readAirportInfo()
        self.indexCities()
        self.indexCountries()

    def get(self, code):
        if not code in self.d:
            self.d[code] = airport(code)
        return self.d[code]

        # Good codes have their airport info and at least one route

    # to a destination which has its airport information complete
    def codesWithRoutes(self, codesWithAirportInfo):
        codes = set([])
        for line in open("routes.dat", encoding='utf-8'):
            l = line.split(",")
            fr = l[2].strip("\"")
            to = l[4].strip("\"")
            if fr in codesWithAirportInfo and to in codesWithAirportInfo:
                codes.add(fr)
                codes.add(to)
        return codes
    # Collect all codes that have in principle all airport information
    def codesWithAirportInfo(self):
        codes = set([])
        for line in open("airports.dat", encoding='utf-8'):
            l = line.split(",")
            if len(l) == 12:
                code = l[4].strip("\"")
                city = l[2].strip("\"")
                country = l[3].strip("\"")
                x = float(l[6])
                y = float(l[7])
                if city and country and x and y:
                    codes.add(code)
        return codes

    def readRoutes(self):
        for line in open("routes.dat", encoding='utf-8'):
            l = line.split(",")
            fr = l[2].strip("\"")
            to = l[4].strip("\"")
            if fr in self.goodCodes and to in self.goodCodes:
                fr_airport = self.get(fr)
                to_airport = self.get(to)
                fr_airport.fly2(to_airport)
                to_airport.fly2(fr_airport)
    def readAirportInfo(self):
        for line in open("airports.dat", encoding='utf-8'):
            l = line.split(",")
            if len(l) == 12:
                code = l[4].strip("\"")
                city = l[2].strip("\"")
                country = l[3].strip("\"")
                x = float(l[6])
                y = float(l[7])
                if code in self.d and code in self.goodCodes:
                    self.d[code].set(city, country, x, y)

    def indexCities(self):
        for code in self.goodCodes:
            c = self.d[code].city
            self.city[c] = self.city.get(c, set([]))
            self.city[c].add(code)

    def indexCountries(self):
        for code in self.goodCodes:
            c = self.d[code].country
            self.country[c] = self.country.get(c, set([]))
            self.country[c].add(code)

    ################################################################
    #
    # Distance between airports from long/lat coordinates on sphere
    # See e.g. http://www.koordinaten.de/informationen/formel.shtml
    #conn
    ################################################################

    def distance(self, fr, to):
        if fr == to: return 0.0
        try:
            return math.acos(
                math.sin(math.radians(fr.x)) * math.sin(math.radians(to.x)) + math.cos(math.radians(fr.x)) * math.cos(
                    math.radians(to.x)) * math.cos(math.radians(to.y - fr.y))) * 6378.137
        except:
            print(fr.code + " " + to.code)
            sys.exit()

    def printPath(self, path):
        for n in path:
            print(n.str())
        dist = 0.0
        for i in range(len(path) - 1):
            dist += self.distance(path[i], path[i + 1])
        print("Distance: %d km" % (dist))

    def suggestAirport(self, s):
        # s is an airport code
        if s in self.d:
            return self.d[s]
        # s is a city, then return random airport code for that city
        elif s in self.city:
            l = list(self.city[s])
            return self.d[l[random.randint(0, len(l) - 1)]]
        # s is a country, then return random airport code for that country
        elif s in self.country:
            l = list(self.country[s])
            return self.d[l[random.randint(0, len(l) - 1)]]
        # return random airport code
        else:
            l = list(self.d.keys())
            return self.d[l[random.randint(0, len(l) - 1)]]

    def suggestNext(self, airport):
        l = list(airport.conn)
        return l[random.randint(0, len(l) - 1)]

In [17]:
ap = airports()
drs = ap.d["DRS"]
print(drs.str()) # DRS, Dresden, Germany
print(drs.conn) # Verbindungen (als Liste von airport Objekten)

for a in drs.conn: # Verbindungen schoen ausgeben
  print(a.str())
  
syd = ap.d["SYD"]
print(syd.str())

print("Distance SYD DRS:")
print(ap.distance(drs,syd))

DRS, Dresden, Germany
{<__main__.airport object at 0x110edbe48>, <__main__.airport object at 0x110d96860>, <__main__.airport object at 0x110d99860>, <__main__.airport object at 0x110ee0ac8>, <__main__.airport object at 0x110edb6d8>, <__main__.airport object at 0x110d9b4e0>, <__main__.airport object at 0x110f21ef0>, <__main__.airport object at 0x110edb710>, <__main__.airport object at 0x110f30320>, <__main__.airport object at 0x110ec5b38>, <__main__.airport object at 0x110edb940>, <__main__.airport object at 0x110edb390>, <__main__.airport object at 0x110ee3780>, <__main__.airport object at 0x110d99f98>, <__main__.airport object at 0x110dd99b0>, <__main__.airport object at 0x110dfa5c0>, <__main__.airport object at 0x110edbf98>, <__main__.airport object at 0x110d717f0>}
CFU, Kerkyra/corfu, Greece
PMI, Palma de Mallorca, Spain
MUC, Munich, Germany
RHO, Rhodos, Greece
BSL, Basel, Switzerland
CGN, Cologne, Germany
LCY, London, United Kingdom
AYT, Antalya, Turkey
NBE, Enfidha, Tunisia
TFS, T

In [18]:
print("German airports")
for a in ap.d.values():
    if a.country == "Germany":
        print(a.str())

# alternativ: funktionelle Programmierung (lambda-expression)
print("---------------------------")
print("\n".join(map(lambda a: a.str(), filter(lambda a: a.country == "Germany", ap.d.values()))))

German airports
DRS, Dresden, Germany
DUS, Duesseldorf, Germany
FDH, Friedrichshafen, Germany
MUC, Munich, Germany
HAM, Hamburg, Germany
TXL, Berlin, Germany
FMM, Memmingen, Germany
CGN, Cologne, Germany
STR, Stuttgart, Germany
HAJ, Hannover, Germany
BRE, Bremen, Germany
HDF, Heringsdorf, Germany
LEJ, Leipzig, Germany
RLG, Laage, Germany
DTM, Dortmund, Germany
NUE, Nuernberg, Germany
FKB, Karlsruhe/Baden-Baden, Germany
FRA, Frankfurt, Germany
SXF, Berlin, Germany
GWT, Westerland, Germany
FMO, Munster, Germany
PAD, Paderborn, Germany
SCN, Saarbruecken, Germany
OAL, Marktoberdorf, Germany
HHN, Hahn, Germany
NRN, Weeze, Germany
LBC, Luebeck, Germany
AGB, Augsburg, Germany
ERF, Erfurt, Germany
KSF, Kassel, Germany
XFW, Hamburg, Germany
MHG, Mannheim, Germany
ZQW, Zweibruecken, Germany
---------------------------
DRS, Dresden, Germany
DUS, Duesseldorf, Germany
FDH, Friedrichshafen, Germany
MUC, Munich, Germany
HAM, Hamburg, Germany
TXL, Berlin, Germany
FMM, Memmingen, Germany
CGN, Cologne, 

In [19]:
airs = ap.d["FRA"].conn
len(airs)

244

In [20]:
allAirportsConnToDrs = ap.d.values()
drs = ap.d["DRS"]
resultAirports = set([])
for a in allAirportsConnToDrs:
    if(ap.distance(drs, a) < 200):
        resultAirports.add(a)
for a in resultAirports:
    print(a.str())

SXF, Berlin, Germany
PRG, Prague, Czech Republic
DRS, Dresden, Germany
PED, Pardubice, Czech Republic
LEJ, Leipzig, Germany
KLV, Karlovy Vary, Czech Republic
ERF, Erfurt, Germany
TXL, Berlin, Germany


In [21]:
verbindungsAnzahlDict = {}
for a in ap.d.values():
    if not a.code in verbindungsAnzahlDict:
        verbindungsAnzahlDict[a.code] = len(ap.d[a.code].conn)
sorted(verbindungsAnzahlDict.items(), key=lambda item:item[1], reverse=True)

[('AMS', 248),
 ('FRA', 244),
 ('CDG', 239),
 ('IST', 234),
 ('ATL', 217),
 ('ORD', 206),
 ('PEK', 203),
 ('MUC', 192),
 ('DME', 188),
 ('DFW', 188),
 ('DXB', 187),
 ('LHR', 172),
 ('DEN', 170),
 ('IAH', 169),
 ('LGW', 165),
 ('BCN', 163),
 ('JFK', 162),
 ('FCO', 162),
 ('MAD', 159),
 ('BRU', 153),
 ('PVG', 153),
 ('STN', 153),
 ('EWR', 153),
 ('LAX', 149),
 ('MAN', 148),
 ('DUS', 147),
 ('YYZ', 147),
 ('CAN', 146),
 ('DUB', 144),
 ('SVO', 144),
 ('CLT', 141),
 ('VIE', 139),
 ('MIA', 138),
 ('ZRH', 137),
 ('DTW', 135),
 ('HKG', 134),
 ('ICN', 133),
 ('LAS', 133),
 ('MSP', 131),
 ('IAD', 127),
 ('PMI', 126),
 ('SIN', 125),
 ('ARN', 124),
 ('PHL', 123),
 ('ORY', 123),
 ('BKK', 122),
 ('CPH', 122),
 ('DOH', 118),
 ('AGP', 117),
 ('MXP', 113),
 ('LED', 112),
 ('KUL', 112),
 ('JED', 110),
 ('TXL', 110),
 ('CTU', 106),
 ('OSL', 105),
 ('SFO', 105),
 ('ATH', 104),
 ('NRT', 104),
 ('TPE', 103),
 ('MCO', 102),
 ('LIS', 102),
 ('BOS', 102),
 ('GVA', 101),
 ('SAW', 98),
 ('DEL', 98),
 ('FLL', 97)

In [47]:
print("Airports with only one connection")
airs = []
for a in ap.d.values():
    if(len(a.conn) == 1):
        airs.append(a)
for a in airs:
    for b in a.conn:
        print(a.city + " connects to " + b.city)

Airports with only one connection
Bodaibo connects to Irkutsk
Ust-Kut connects to Irkutsk
Ayacucho connects to Lima
Huánuco connects to Lima
San Cristóbal connects to Guayaquil
Arvidsjaur connects to Lycksele
Ornskoldsvik connects to Stockholm
Hemavan connects to Vilhelmina
Old Harbor connects to Kodiak
Zachar Bay connects to Amook Bay
Karluk connects to Larsen Bay
Port Williams connects to Seal Bay
Basco connects to Manila
Cotabato connects to Manila
Catarman connects to Manila
Roxas City connects to Manila
Tuguegarao connects to Manila
Jonesboro connects to St. Louis
Metakatla connects to Ketchikan
Kangiqsualujjuaq connects to Quujjuaq
Meridian connects to Atlanta
Muscle Shoals connects to Atlanta
Hattiesburg/Laurel connects to Atlanta
Alice Town connects to Fort Lauderdale
Treasure Cay connects to Fort Lauderdale
Greenville connects to Tupelo
Daocheng connects to Chengdu
Nikolai connects to Mcgrath
Barisal connects to Dhaka
Cox's Bazar connects to Dhaka
Jessore connects to Dhaka
Gri

In [64]:
print("The airport which hat farest distance from DRS")
drs = ap.get('DRS')
farest_airports = sorted(list(ap.d.values()), key=lambda a:ap.distance(a,drs), reverse=True)
for a in farest_airports:
    print(a.str() + " distance: " + str(ap.distance(a, drs)))

The airport which hat farest distance from DRS
CHC, Christchurch, New Zealand distance: 18232.08208732829
WLG, Wellington, New Zealand distance: 18223.331229573592
DUD, Dunedin, New Zealand distance: 18215.310776771446
PPQ, Paraparaumu, New Zealand distance: 18201.740394524182
TIU, Timaru, New Zealand distance: 18195.64765185898
PMR, Palmerston North, New Zealand distance: 18192.261087028604
NPE, NAPIER, New Zealand distance: 18191.896500425406
GIS, Gisborne, New Zealand distance: 18180.743172937702
BHE, Woodbourne, New Zealand distance: 18180.350899603112
WAG, Wanganui, New Zealand distance: 18128.716089395788
NSN, Nelson, New Zealand distance: 18123.64855128041
IVC, Invercargill, New Zealand distance: 18108.461834754446
TUO, Taupo, New Zealand distance: 18086.36220473139
HKK, Hokitika, New Zealand distance: 18079.606500486327
ZQN, Queenstown International, New Zealand distance: 18067.262410763517
WHK, Whakatane, New Zealand distance: 18058.951239504553
WSZ, Westport, New Zealand dist

ZAH, Zahedan, Iran distance: 4564.03614560932
MSW, Massawa, Eritrea distance: 4559.651506210124
AZI, Abu Dhabi, United Arab Emirates distance: 4559.255167435949
TJU, Kulyab, Tajikistan distance: 4556.28480051647
FRU, Bishkek, Kyrgyzstan distance: 4552.381612721915
SHJ, Sharjah, United Arab Emirates distance: 4549.081325384406
AZN, Andizhan, Uzbekistan distance: 4548.6922421025565
DXB, Dubai, United Arab Emirates distance: 4545.775348418326
KAD, Kaduna, Nigeria distance: 4539.608888048676
FEG, Fergana, Uzbekistan distance: 4534.460890182445
OUA, Ouagadougou, Burkina Faso distance: 4533.0631923649
MVR, Maroua, Cameroon distance: 4528.850935992793
YXP, Pangnirtung, Canada distance: 4527.912160916016
KHS, Khasab, Oman distance: 4524.146116069141
TOF, Tomsk, Russia distance: 4522.725034500804
PLX, Semiplatinsk, Kazakhstan distance: 4518.578763564525
NKC, Nouakschott, Mauritania distance: 4513.378428575255
YCY, Clyde River, Canada distance: 4492.529934400055
NMA, Namangan, Uzbekistan distanc

In [67]:
print("Random walker")
a = ap.suggestAirport("")
for i in range(10):
    print(a.str())
    a = ap.suggestNext(a)
print(a.str())

Random walker
HND, Tokyo, Japan
KMQ, Kanazawa, Japan
HND, Tokyo, Japan
HAC, Hachijojima, Japan
HND, Tokyo, Japan
SHM, Nanki-shirahama, Japan
HND, Tokyo, Japan
OKJ, Okayama, Japan
TPE, Taipei, Taiwan
SGN, Ho Chi Minh City, Vietnam
SYD, Sydney, Australia


In [79]:
import random
print("Random walker ranking")
a = ap.suggestAirport("")
visits = {}
for i in range(100000):
    visits[a.code] = visits.get(a.code, 0) + 1
    if random.uniform(0,1) < 0.15:
        a = ap.suggestAirport("")
    else:
        a = ap.suggestNext(a)
# Sortieren der besuchen Flughäfen nach Häufigkeit
visited_airports = sorted(visits.items(), key=lambda x: x[1], reverse=True)
for (k, v) in visited_airports[:10]:
#     print("Visited %s %d times" % (ap.d[k].str(), v))
#     print("Visited " + ap.d[k].str() +" " + str(v) + " times")
    print("Visited %s %d times." % (ap.d[k].str(),v))

Random walker ranking
Visited IST, Istanbul, Turkey 468 times.
Visited ATL, Atlanta, United States 459 times.
Visited ORD, Chicago, United States 456 times.
Visited DFW, Dallas-Fort Worth, United States 437 times.
Visited DEN, Denver, United States 433 times.
Visited AMS, Amsterdam, Netherlands 431 times.
Visited CDG, Paris, France 393 times.
Visited IAH, Houston, United States 386 times.
Visited DME, Moscow, Russia 384 times.
Visited FRA, Frankfurt, Germany 379 times.


In [72]:
import random
for i in range(100):
    print(str(random.uniform(0,1)))

0.6812987441735395
0.10495097384329688
0.6043118952496759
0.14463354641745274
0.6359495150449858
0.3041989663348408
0.7959647118914747
0.934808856874355
0.301367075698436
0.48024252299324055
0.17744027010255992
0.9527187793271297
0.6811177687212358
0.10167196836567238
0.3518806877695718
0.8323348079799732
0.7329657407704954
0.5222993955187883
0.7184172274531769
0.015279200658865122
0.6899338750385635
0.8823250333556734
0.10196435947200533
0.7019248054658819
0.4131924187518382
0.015560729896314629
0.8484920298022521
0.7409116996430593
0.1988568117692534
0.49043242338815063
0.8247261198225954
0.3933192037289638
0.1807543272269011
0.15671311941176969
0.86510837733613
0.29766934966967273
0.5638410061947846
0.699635762810173
0.38757816347365326
0.5783991297192956
0.10458607106258733
0.9606143658553532
0.7641885247130763
0.7291458246280257
0.5113523464297728
0.7965585630520976
0.4424093515676625
0.9180404243846463
0.41149591369733485
0.9015852861059632
0.8423060745079515
0.45855629689414756


DRS, Dresden, Germany
